## Block Shear in Flange-Connect Tee Sections - All Tee Sections

![Tee Block Shear Cases](tee-block-shear.svg)

This example calculates the block shear resistance (Section 13.11) for all Tee sections as tension members
bolted only through the flange.  Bolts are chosend such that there are either $N=2$ or $N=4$ bolts across the flange,
and that there are sufficient numbers of lines of bolts ($M$) s.t. they can develop the full Tee tension resistance, 
$T_r = \min(0.75 A_{ne} F_u,0.9 A_g F_y)$ in double shear with threads excluded.  $M_{min} = 2$.

In [3]:
from Designer import sst
from Designer import show
SST = sst.SST()

#### Symbols and nomenclature:

![Dimensions and Symbols](tee-dimens.svg)

In [4]:
Fy = 350.
Fu = 450.

def Tr_13p11(An,Agv,Ut,Fy=Fy,Fu=Fu,phiu=0.75):
    """Compute Tr as per S16-09 13.11"""
    if Fy > 485:
        Fv = Fy
    else:
        Fv = (Fy+Fu)/2.
    return phiu*(Ut*An*Fu + 0.6*Agv*Fv)/1000.

def usual_gauge_W(b):
    if b < 130.:
        return 60.
    elif b < 150.:
        return 80.
    elif b < 190.:
        return 100.
    else:
        return 130.

### Detailed symbols
The following shows the cross-sections for 6 of the 7 failure cases.  The light shaded area shows the tension fracture surface, and the black areas represent the bolt holes.  To compute the tension area, the white and black areas are subtracted from the full cross-sectional area of the Tee (except for Case 1).  This technique more accurately includes the effect of the stem and the fillet.

$g$ is the gauge of the inner bolts, and $G$ is the gauge of the outer bolts.  When there are only 2 bolts
across, these are the same: $g=G$.

![Dimensions and Symbols](tee-An-details.svg)

### Compute for one Tee section
This function computes the resistance of the 7 cases for one Tee section.  Input arguments are the values required from the section tables.  It computes the ratio of $T_r$ (block shear) to $T_r$ (gross or net area).

In [5]:
def onetee(A,d,b,t,w,k):
    M = 2            # number of lines of bolts
    s = 75.          # pitch - two lines of bolts
    L = (M-1)*s      # length of connection
    e = 30.          # end distance
    min_edge_dist = 30.
    db = 24.         # bolt diameter
    hd = db + 2. + 2.  # hole allowance
    min_g = 2.7*db   # minimum pitch
    
    extra_g = [50.,60.,70.,80.,85.,90.]    # extra gauges for wider flanges
    g = usual_gauge_W(b)
    
    # place 4 bolts across the flange, if possible, using the largest extra gauge    
    N = 0
    for i,eg in enumerate(extra_g):
        if g + 2.*eg + 2.*min_edge_dist > b:
            if i == 0:   # no room for 4
                N = 2
                G = g
                break
            N = 4
            G = g + 2.*eg
            break
    if N == 0:
        N = 4
        G = g + 2.*extra_g[-1]
 
    # compute the number of bolts required to develop Tr
    if 1:
        Tr = min(0.9*Fy*A/1000.,0.75*(0.85*(A-N*hd*t)*Fu/1000.))
        Vr = 2*0.6*0.8*1.*1.*(3.14159*db*db/4.)*830./1000.   # A325M bolts
        M = max(2,int((Tr/(N*Vr)) + 0.999))
        L = (M-1)*s        
        
    ##print g,N,G,M,L
    
    data = {'N':N, 'M':M}
    
    ## Case 1
    An = (b-(g+hd))*t - (N-2)*hd*t
    Agv = 2.*(e+L)*t
    data['Case 1'] = Tr_13p11(An,Agv,0.9)
    
    ## Case 2
    An = A - (d-k)*w - N*hd*t
    Agv = (e+L)*w
    data['Case 2'] = Tr_13p11(An,Agv,1.0)
    
    ## Case 3
    An = A - (d-k)*w - (b-G)*t - (N-1)*hd*t
    Agv = (e+L)*w + 2.*(e+L)*t
    data['Case 3'] = Tr_13p11(An,Agv,1.0)
    
    ## Case 4
    An = A - (b-G)*t - (N-1)*hd*t
    Agv = 2.*(e+L)*t
    data['Case 4'] = Tr_13p11(An,Agv,0.6)
    
    ## Case 5
    An = A - (d-k)*w - (b/2.-G/2.)*t - (N-0.5)*hd*t
    Agv = (e+L)*t + (e+L)*w
    data['Case 5'] = Tr_13p11(An,Agv,0.6)
    
    ## Case 6
    An = A - (b/2.-G/2.)*t - (N-0.5)*hd*t
    Agv = (e+L)*t
    data['Case 6'] = Tr_13p11(An,Agv,0.6)
    
    ## Case 7
    An = 0.
    Agv = 4.*(e+L)*t
    data['Case 7'] = Tr_13p11(An,Agv,1.0)
    
    # compute ratios to Tr for gross/net area
    if 1:
        for c in data.keys():
            if c.startswith('Case '):
                data[c] = round(data[c]/Tr,3)
    
    return data

In [6]:
# test the function on one specific Tee

A,d,b,t,w,k = SST.section('WT345x85',properties='A,D,B,T,W,K')
show('A,d,b,t,w,k')
onetee(A,d,b,t,w,k)

A = 1.08e+04
d = 347
b = 256
t = 23.6
w = 14.5
k = 45


{'Case 1': 1.055,
 'Case 2': 0.878,
 'Case 3': 1.387,
 'Case 4': 1.33,
 'Case 5': 0.937,
 'Case 6': 1.018,
 'Case 7': 1.594,
 'M': 4,
 'N': 2}

### Compute for all Tees

Now calculate for all 7 cases for all WT sections.

In [10]:
# callback for SST .select() method - interfaces to 'onetee()'
def fn(Dsg,A,D,B,T,W,K):
    #Dsg,A,d,b,t,w,k = [sect[x] for x in 'Dsg,A,D,B,T,W,K'.split(',')]
    return onetee(A,D,B,T,W,K)

CASES = ['Case 1','Case 2','Case 3','Case 4','Case 5','Case 6','Case 7']

wts = SST.section_table('WT').sort_values(by=['Mass'],ascending=False)
raw = SST.select(wts,fn,maxn=None)[['Dsg','N','M']+CASES]
raw.head(5)

,Dsg,N,M,Case 1,Case 2,Case 3,Case 4,Case 5,Case 6,Case 7
Dsg,,,,,,,,,,
WT180x543,WT180x543,4,12,3.000,1.721,3.838,2.946,2.455,1.826,4.851
WT180x495,WT180x495,4,10,2.593,1.594,3.324,2.554,2.134,1.630,4.052
WT180x450,WT180x450,4,10,2.616,1.598,3.369,2.591,2.157,1.648,4.111
WT180x409,WT180x409,4,9,2.409,1.535,3.117,2.397,1.998,1.551,3.710
WT180x372,WT180x372,4,8,2.202,1.468,2.859,2.202,1.836,1.454,3.307


### Display the result, showing the governing case for each

In [11]:
# PrettyTable from https://github.com/mgymrek/itable
# sudo pip install itable

import itable as it

In [12]:
pt = it.PrettyTable(raw,tstyle=it.TableStyle(theme="theme1"), center=True)
cs = it.CellStyle()
cs.set("background-color", "pink")
cs.set("color", "purple")
cs.set("font-weight","bold")

In [13]:
# Search each row, colouring the cell with the min value
for i,(ind,row) in enumerate(raw.iterrows()):
    small = 1.0E12
    for c in CASES:
        if row[c] < small:
            small = row[c]
            k = raw.columns.get_loc(c)
    pt.set_cell_style(style=cs,rows=[i,],cols=[k,])
pt

Dsg,N,M,Case 1,Case 2,Case 3,Case 4,Case 5,Case 6,Case 7
WT180x543,4,12,3.0,1.721,3.838,2.946,2.455,1.826,4.851
WT180x495,4,10,2.593,1.594,3.324,2.554,2.134,1.63,4.052
WT180x450,4,10,2.616,1.598,3.369,2.591,2.157,1.648,4.111
WT180x409,4,9,2.409,1.535,3.117,2.397,1.998,1.551,3.71
WT180x372,4,8,2.202,1.468,2.859,2.202,1.836,1.454,3.307
WT180x338.5,4,7,1.983,1.401,2.589,1.995,1.666,1.35,2.882
WT180x317,4,7,2.002,1.402,2.619,2.021,1.679,1.363,2.924
WT180x296,4,6,1.771,1.332,2.331,1.8,1.5,1.253,2.474
WT180x275.5,4,6,1.778,1.333,2.35,1.815,1.509,1.261,2.496
WT180x254.5,4,6,1.777,1.334,2.358,1.821,1.514,1.264,2.502
